In [1]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union
from pydantic.config import ConfigDict

from src.utilities import *

import instructor
from llama_cpp import Llama



In [ ]:
pd.read_excel

In [2]:
csv_path = 'archives/www.darty.com.csv'
feedbacks_df = pd.read_excel(csv_path)
feedbacks_df['Date'] = pd.to_datetime(feedbacks_df['Date'], format='%B %d, %Y', errors='coerce')

feedbacks_df.head()

,Title,Date,Content,Page,Username,Total reviews,Location,Rating
0,"Fuyez, une arnaque",2024-02-05,2000€ d’appareils et ils ne fonctionnent pas p...,1,Violaine Becha,1 review,FR,1.0
1,Centrale vapeur calor défaillante au bout de 5...,2024-02-03,Le lundi 29 janvier 2024 je récupère une centr...,1,Isabelle bas-rhin,9 reviews,FR,1.0
2,Évitez !! Les actions du vendeur sont absolume...,2023-03-30,Évitez d'acheter auprès de ce vendeur en ligne...,1,Mikhail Shangin,3 reviews,FR,1.0
3,Le SAV est une catastrophe!,2024-01-15,Le SAV est une catastrophe!Le client ne l'est ...,1,SCD,1 review,FR,1.0
4,Machine a laver Bosch,2024-01-05,Commandée sur le site de Darty machine a laver...,1,Christiane,56 reviews,FR,5.0


In [4]:
grouped_data = feedbacks_df.groupby(feedbacks_df['Date'].dt.year)
mean_rating = feedbacks_df.groupby(feedbacks_df['Date'].dt.year).Rating.mean()
group_count = grouped_data.Title.count()

x = np.array(mean_rating.index)
y = group_count
c = mean_rating

In [5]:
import plotly.express as px

# Assuming your arrays are named x, y, and c
fig = px.scatter(x=x, y=y, color=c)
fig.show()


In [6]:
from scipy.interpolate import interp1d

x_new = np.linspace(x.min(), x.max(),500)

y_smooth = interp1d(x, y, kind='quadratic')(x_new)
c_smooth = interp1d(x, c, kind='quadratic')(x_new)


fig = px.scatter(x=x_new, y=y_smooth, color=c_smooth)
fig.show()



In [9]:
np.datetime64('NaT')

numpy.datetime64('NaT')

In [10]:
feedbacks_df[feedbacks_df['Date'] == np.datetime64('NaT')]

,Title,Date,Content,Page,Username,Total reviews,Location,Rating


In [ ]:
bubble_client.create(
    "Feedback",
    [{
        "Company": COMPANY_ID,
        "Content": row[],
        "Name": sub_categorie.name,
        "Project": PROJECT_ID,
        "Description": categorie.description,
    } for row in feedbacks_df.iterrows() )